Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score

import plotly.express as px

Merge all datasets

In [5]:
data_apr = pd.read_csv("uber-raw-data-apr14.csv")
data_aug = pd.read_csv("uber-raw-data-aug14.csv")
data_jul = pd.read_csv("uber-raw-data-jul14.csv")
data_jun = pd.read_csv("uber-raw-data-jun14.csv")
data_may = pd.read_csv("uber-raw-data-may14.csv")
data_sep = pd.read_csv("uber-raw-data-sep14.csv")
uber_fin = pd.concat([data_apr,data_aug, data_jul, data_jun, data_may,data_sep]).reset_index(drop='index')

In [6]:
uber_fin.shape


(4534327, 4)

In [7]:
uber_fin.head()


,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


Datetime

In [8]:
uber_fin['Pickup_time'] = pd.to_datetime(uber_fin['Date/Time'])


In [9]:
uber_fin['Pickup_time']


0         2014-04-01 00:11:00
1         2014-04-01 00:17:00
2         2014-04-01 00:21:00
3         2014-04-01 00:28:00
4         2014-04-01 00:33:00
                  ...        
4534322   2014-09-30 22:57:00
4534323   2014-09-30 22:57:00
4534324   2014-09-30 22:58:00
4534325   2014-09-30 22:58:00
4534326   2014-09-30 22:58:00
Name: Pickup_time, Length: 4534327, dtype: datetime64[ns]

Extracting Month, Day of the week from Pickup_time

In [10]:
uber_fin['Month'] = uber_fin['Pickup_time'].apply(lambda x : x.month)
uber_fin['Week_day'] = uber_fin['Pickup_time'].apply(lambda x : x.weekday())

In [11]:
uber_fin

,Date/Time,Lat,Lon,Base,Pickup_time,Month,Week_day
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,2014-04-01 00:11:00,4,1
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,2014-04-01 00:17:00,4,1
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,2014-04-01 00:21:00,4,1
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,2014-04-01 00:28:00,4,1
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,2014-04-01 00:33:00,4,1
...,...,...,...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764,2014-09-30 22:57:00,9,1
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764,2014-09-30 22:57:00,9,1
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764,2014-09-30 22:58:00,9,1
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764,2014-09-30 22:58:00,9,1


Extract samples from the dataset

In [12]:
uber_sep = uber_fin.sample(10000, random_state=42).reset_index(drop='index')
uber_sep

,Date/Time,Lat,Lon,Base,Pickup_time,Month,Week_day
0,4/10/2014 20:15:00,40.7588,-73.9726,B02617,2014-04-10 20:15:00,4,3
1,6/22/2014 14:48:00,40.8451,-73.9418,B02617,2014-06-22 14:48:00,6,6
2,8/28/2014 22:58:00,40.7399,-73.9823,B02764,2014-08-28 22:58:00,8,3
3,4/28/2014 16:30:00,40.6449,-73.7824,B02682,2014-04-28 16:30:00,4,0
4,9/17/2014 20:40:00,40.7636,-73.9798,B02598,2014-09-17 20:40:00,9,2
...,...,...,...,...,...,...,...
9995,9/29/2014 9:29:00,40.7182,-73.9856,B02682,2014-09-29 09:29:00,9,0
9996,5/30/2014 12:55:00,40.7695,-73.9919,B02682,2014-05-30 12:55:00,5,4
9997,5/23/2014 5:16:00,40.7594,-73.9164,B02682,2014-05-23 05:16:00,5,4
9998,8/3/2014 11:54:00,40.7803,-74.0075,B02598,2014-08-03 11:54:00,8,6


Drop useless column

In [13]:
uber_sep = uber_sep.drop(['Date/Time', 'Base', 'Pickup_time'], axis=1)


Preprocessing on one sample

In [ ]:
# Preprocessing on sample

In [14]:
num_features = uber_sep.columns.tolist()
cat_features = [num_features.pop(num_features.index('Month'))]
cat_features.append(num_features.pop(num_features.index('Week_day')))
cat_features

['Month', 'Week_day']

In [15]:
num_features


['Lat', 'Lon']

In [16]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first'))
    ])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [18]:
us_preprocessed = preprocessor.fit_transform(uber_sep)


In [19]:
us_preprocessed


<10000x13 sparse matrix of type '<class 'numpy.float64'>'
	with 37527 stored elements in Compressed Sparse Row format>

Kmeans on sample
Elbow on sample

In [20]:
wcss =  []
k = []
for i in range (1,11): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(us_preprocessed)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print(f"WCSS for K={i} --> {(wcss[-1])}")

wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)
fig.show()

WCSS for K=1 --> 34651.695699999975
WCSS for K=2 --> 29140.93803629097
WCSS for K=3 --> 24555.47301773855
WCSS for K=4 --> 22879.210410850894
WCSS for K=5 --> 21689.93285563315
WCSS for K=6 --> 20329.43108818984
WCSS for K=7 --> 19267.085975026966
WCSS for K=8 --> 18211.748833089925
WCSS for K=9 --> 16863.087418088595
WCSS for K=10 --> 15883.536158857478


Silouhette on sample

In [21]:
sil = []
k = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(us_preprocessed)
    sil.append(silhouette_score(us_preprocessed, kmeans.predict(us_preprocessed)))
    k.append(i)
    print(f"Silhouette score for K={i} is {sil[-1]}")

cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

fig = px.bar(data_frame=cluster_scores,  
             x=k, 
             y=cluster_scores.iloc[:, -1]
            )
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

fig.show()

Silhouette score for K=2 is 0.36497214147819523
Silhouette score for K=3 is 0.14960810117721732
Silhouette score for K=4 is 0.1462970695434406
Silhouette score for K=5 is 0.12246646441817687
Silhouette score for K=6 is 0.13020381939262574
Silhouette score for K=7 is 0.15305294067553984
Silhouette score for K=8 is 0.19866657927764614
Silhouette score for K=9 is 0.2060884052249794
Silhouette score for K=10 is 0.2084176762161159


In [22]:
kmeans = KMeans(n_clusters=4 , random_state = 0)
kmeans.fit(us_preprocessed)

KMeans(n_clusters=4, random_state=0)

In [24]:
uber_km = uber_sep.copy()


In [25]:
uber_km['Cluster_KMeans'] = kmeans.labels_
uber_km

,Lat,Lon,Month,Week_day,Cluster_KMeans
0,40.7588,-73.9726,4,3,3
1,40.8451,-73.9418,6,6,1
2,40.7399,-73.9823,8,3,3
3,40.6449,-73.7824,4,0,2
4,40.7636,-73.9798,9,2,3
...,...,...,...,...,...
9995,40.7182,-73.9856,9,0,0
9996,40.7695,-73.9919,5,4,3
9997,40.7594,-73.9164,5,4,3
9998,40.7803,-74.0075,8,6,3


In [26]:
fig = px.scatter_mapbox(uber_km.sort_values('Week_day'), lat="Lat", lon="Lon", color="Cluster_KMeans",
                        mapbox_style="open-street-map",
                        animation_frame = 'Week_day',
                        zoom=9,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

DBSCAN on sample
Euclidian metric

In [27]:
db_euc = DBSCAN(eps=0.4, min_samples=90, metric="euclidean", algorithm="brute")
db_euc.fit(us_preprocessed)

DBSCAN(algorithm='brute', eps=0.4, min_samples=90)

In [29]:
uber_km['Cluster_db_euc'] = db_euc.labels_
tk = (uber_km['Cluster_db_euc'] != -1)
uber_kmf = uber_km.loc[tk,:]
uber_kmf['Cluster_db_euc'].value_counts()

5     235
10    231
6     227
13    218
8     202
1     200
7     190
11    184
14    170
4     166
0     156
15    147
9     141
12    133
3     133
2     114
16    107
Name: Cluster_db_euc, dtype: int64

In [30]:
fig = px.scatter_mapbox(uber_kmf.sort_values('Week_day'), lat="Lat", lon="Lon", color="Cluster_db_euc",
                        mapbox_style="open-street-map",
                        animation_frame = 'Week_day',
                        zoom=10,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

More advanced feature selection
Feature selection on total dataset

Extracting Hour, Rounded coordinates and classes of 30 min range

In [33]:
uber_fin['Hour'] = uber_fin['Pickup_time'].apply(lambda x : x.hour)
uber_fin['Lat_r'] = uber_fin['Lat'].apply(lambda x : round(x,2))
uber_fin['Lon_r'] = uber_fin['Lon'].apply(lambda x : round(x,2))
uber_fin['Bin_30'] = uber_fin['Pickup_time'].dt.floor("30min")
uber_fin['Bin_30'] = uber_fin['Bin_30'].apply(lambda x : x.strftime('%H:%M'))

In [34]:
uber_fin

,Date/Time,Lat,Lon,Base,Pickup_time,Month,Week_day,Hour,Lat_r,Lon_r,Bin_15,Bin_30
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,2014-04-01 00:11:00,4,1,0,40.77,-73.95,00:00,00:00
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,2014-04-01 00:17:00,4,1,0,40.73,-74.03,00:15,00:00
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,2014-04-01 00:21:00,4,1,0,40.73,-73.99,00:15,00:00
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,2014-04-01 00:28:00,4,1,0,40.76,-73.98,00:15,00:00
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,2014-04-01 00:33:00,4,1,0,40.76,-73.97,00:30,00:30
...,...,...,...,...,...,...,...,...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764,2014-09-30 22:57:00,9,1,22,40.77,-73.98,22:45,22:30
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764,2014-09-30 22:57:00,9,1,22,40.69,-74.18,22:45,22:30
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764,2014-09-30 22:58:00,9,1,22,40.85,-73.93,22:45,22:30
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764,2014-09-30 22:58:00,9,1,22,40.71,-74.01,22:45,22:30


Creating a column with number of pickups giving a 30min class in a month

In [35]:
uber_fin.groupby(['Month','Bin_30'])['Pickup_time'].count().reset_index(name='Pickups')


,Month,Bin_30,Pickups
0,4,00:00,6720
1,4,00:30,5190
2,4,01:00,4328
3,4,01:30,3441
4,4,02:00,2658
...,...,...,...
283,9,21:30,29623
284,9,22:00,27469
285,9,22:30,24348
286,9,23:00,20153


In [36]:
uber_fin['Pickups'] = uber_fin.groupby(['Month','Bin_30'])['Pickup_time'].transform('count')


In [37]:
uber_fin


,Date/Time,Lat,Lon,Base,Pickup_time,Month,Week_day,Hour,Lat_r,Lon_r,Bin_15,Bin_30,Pickups
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,2014-04-01 00:11:00,4,1,0,40.77,-73.95,00:00,00:00,6720
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,2014-04-01 00:17:00,4,1,0,40.73,-74.03,00:15,00:00,6720
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,2014-04-01 00:21:00,4,1,0,40.73,-73.99,00:15,00:00,6720
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,2014-04-01 00:28:00,4,1,0,40.76,-73.98,00:15,00:00,6720
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,2014-04-01 00:33:00,4,1,0,40.76,-73.97,00:30,00:30,5190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764,2014-09-30 22:57:00,9,1,22,40.77,-73.98,22:45,22:30,24348
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764,2014-09-30 22:57:00,9,1,22,40.69,-74.18,22:45,22:30,24348
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764,2014-09-30 22:58:00,9,1,22,40.85,-73.93,22:45,22:30,24348
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764,2014-09-30 22:58:00,9,1,22,40.71,-74.01,22:45,22:30,24348


Sample dataset

In [44]:
uber_sep2 = uber_fin.sample(10000, random_state=42).reset_index(drop='index')
uber_sep2

,Date/Time,Lat,Lon,Base,Pickup_time,Month,Week_day,Hour,Lat_r,Lon_r,Bin_15,Bin_30,Pickups
0,4/10/2014 20:15:00,40.7588,-73.9726,B02617,2014-04-10 20:15:00,4,3,20,40.76,-73.97,20:15,20:00,17724
1,6/22/2014 14:48:00,40.8451,-73.9418,B02617,2014-06-22 14:48:00,6,6,14,40.85,-73.94,14:45,14:30,17928
2,8/28/2014 22:58:00,40.7399,-73.9823,B02764,2014-08-28 22:58:00,8,3,22,40.74,-73.98,22:45,22:30,21872
3,4/28/2014 16:30:00,40.6449,-73.7824,B02682,2014-04-28 16:30:00,4,0,16,40.64,-73.78,16:30,16:30,21423
4,9/17/2014 20:40:00,40.7636,-73.9798,B02598,2014-09-17 20:40:00,9,2,20,40.76,-73.98,20:30,20:30,31452
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9/29/2014 9:29:00,40.7182,-73.9856,B02682,2014-09-29 09:29:00,9,0,9,40.72,-73.99,09:15,09:00,19693
9996,5/30/2014 12:55:00,40.7695,-73.9919,B02682,2014-05-30 12:55:00,5,4,12,40.77,-73.99,12:45,12:30,12019
9997,5/23/2014 5:16:00,40.7594,-73.9164,B02682,2014-05-23 05:16:00,5,4,5,40.76,-73.92,05:15,05:00,4593
9998,8/3/2014 11:54:00,40.7803,-74.0075,B02598,2014-08-03 11:54:00,8,6,11,40.78,-74.01,11:45,11:30,15752


Dropping useless columns

In [45]:
uber_sep2 = uber_sep2.drop(['Date/Time', 'Lat', 'Lon', 'Base', 'Pickup_time','Bin_15'], axis=1)


In [46]:
uber_sep2

,Month,Week_day,Hour,Lat_r,Lon_r,Bin_30,Pickups
0,4,3,20,40.76,-73.97,20:00,17724
1,6,6,14,40.85,-73.94,14:30,17928
2,8,3,22,40.74,-73.98,22:30,21872
3,4,0,16,40.64,-73.78,16:30,21423
4,9,2,20,40.76,-73.98,20:30,31452
...,...,...,...,...,...,...,...
9995,9,0,9,40.72,-73.99,09:00,19693
9996,5,4,12,40.77,-73.99,12:30,12019
9997,5,4,5,40.76,-73.92,05:00,4593
9998,8,6,11,40.78,-74.01,11:30,15752


Preprocessing

In [47]:
num_features = uber_sep2.columns.tolist()
cat_features = [num_features.pop(num_features.index('Month'))]
cat_features.append(num_features.pop(num_features.index('Week_day')))
cat_features.append(num_features.pop(num_features.index('Hour')))
cat_features.append(num_features.pop(num_features.index('Bin_30')))
cat_features

['Month', 'Week_day', 'Hour', 'Bin_30']

In [48]:
num_features


['Lat_r', 'Lon_r', 'Pickups']

In [49]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first'))
    ])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [50]:
us_preprocessed = preprocessor.fit_transform(uber_sep2)


In [51]:
us_preprocessed


<10000x84 sparse matrix of type '<class 'numpy.float64'>'
	with 67185 stored elements in Compressed Sparse Row format>

Kmeans on sample
Elbow on sample

In [52]:
wcss =  []
k = []
for i in range (1,11): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(us_preprocessed)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print(f"WCSS for K={i} --> {(wcss[-1])}")

wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)
fig.show()

WCSS for K=1 --> 63550.59649999999
WCSS for K=2 --> 56446.62273947104
WCSS for K=3 --> 50948.58775079281
WCSS for K=4 --> 47849.911812762795
WCSS for K=5 --> 45967.394432428606
WCSS for K=6 --> 44348.8848213916
WCSS for K=7 --> 43087.0076081366
WCSS for K=8 --> 42367.315994837234
WCSS for K=9 --> 41608.54304164697
WCSS for K=10 --> 40605.99931604302


Silhouette on sample

In [53]:
sil = []
k = []
for i in range (2,11): 
    kmeans = KMeans(n_clusters= i, random_state = 0)
    kmeans.fit(us_preprocessed)
    sil.append(silhouette_score(us_preprocessed, kmeans.predict(us_preprocessed)))
    k.append(i)
    print(f"Silhouette score for K={i} is {sil[-1]}")

cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

fig = px.bar(data_frame=cluster_scores,  
             x=k, 
             y=cluster_scores.iloc[:, -1]
            )
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

fig.show()

Silhouette score for K=2 is 0.11095359234192527
Silhouette score for K=3 is 0.12178651710128267
Silhouette score for K=4 is 0.0978846102805838
Silhouette score for K=5 is 0.08626071094074493
Silhouette score for K=6 is 0.08945432816655692
Silhouette score for K=7 is 0.09375808606789957
Silhouette score for K=8 is 0.06900053122975389
Silhouette score for K=9 is 0.07501680016844
Silhouette score for K=10 is 0.07774818934114808


In [54]:
kmeans = KMeans(n_clusters=3 , random_state = 0)
kmeans.fit(us_preprocessed)

KMeans(n_clusters=3, random_state=0)

In [55]:
uber_km2 = uber_sep2.copy()


In [56]:
uber_km2['Cluster_KMeans'] = kmeans.labels_
uber_km2

,Month,Week_day,Hour,Lat_r,Lon_r,Bin_30,Pickups,Cluster_KMeans
0,4,3,20,40.76,-73.97,20:00,17724,2
1,6,6,14,40.85,-73.94,14:30,17928,2
2,8,3,22,40.74,-73.98,22:30,21872,0
3,4,0,16,40.64,-73.78,16:30,21423,1
4,9,2,20,40.76,-73.98,20:30,31452,0
...,...,...,...,...,...,...,...,...
9995,9,0,9,40.72,-73.99,09:00,19693,2
9996,5,4,12,40.77,-73.99,12:30,12019,2
9997,5,4,5,40.76,-73.92,05:00,4593,2
9998,8,6,11,40.78,-74.01,11:30,15752,2


In [57]:
fig = px.scatter_mapbox(uber_km2.sort_values('Bin_30'), lat="Lat_r", lon="Lon_r", color="Cluster_KMeans",
                        mapbox_style="open-street-map",
                        animation_frame = 'Bin_30',
                        size='Pickups',
                        color_continuous_scale=px.colors.cyclical.HSV,
                        zoom=10,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()

DBSCAN on sample
Euclidian metric

In [58]:
db_euc = DBSCAN(eps=2, min_samples=12, metric="euclidean", algorithm="brute")
db_euc.fit(us_preprocessed)

DBSCAN(algorithm='brute', eps=2, min_samples=12)

In [59]:
uber_km2['Cluster_db_euc'] = db_euc.labels_
tk = (uber_km2['Cluster_db_euc'] != -1)
uber_kmf2 = uber_km2.loc[tk,:]
uber_kmf2['Cluster_db_euc'].value_counts()

0    8755
1     712
2      80
Name: Cluster_db_euc, dtype: int64

In [63]:
fig = px.scatter_mapbox(uber_km2.sort_values('Bin_30'), lat="Lat_r", lon="Lon_r", color="Cluster_db_euc",
                        mapbox_style="open-street-map",
                        animation_frame = 'Bin_30',
                        size='Pickups',
                        color_continuous_scale=px.colors.cyclical.Edge,
                        zoom=10,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()
